# Trade Execution Simulator (Strategy Loop)

**Jonathan Gil**

**Last Updated: October 26, 2025**

Goal: Simulate a simple trading rule and order execution.

Task:
-  Create ExecutionLoop.ipynb. ✅
-  For one ticker, implement a basic rule:
-  “Buy when 20-day SMA > 50-day SMA, sell when <.”
-  Track positions (1 or 0) and portfolio value over time.
-  Output a trade log (Date, Action, Price, Position, PortfolioValue).

Outcome: a basic but real trading simulator.

In [2]:
# IMPORT STATEMENTS
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'yfinance'

In [ ]:
# PARAMETERS

# This is the stock I want to simulate trading -- AAPL/TSLA/NVDA/etc.
TICKER = "NVDA" 
START_DATE = "2010-12-27"
# 'None' means up to the current date
END_DATE = "2025-10-27"#"2023-12-27" 
# 'Adj Close' means adjusted for dividends/splits
PRICE_COL = "Adj Close"

# Amount of money to start with (default = $1,000)
STARTING_CAPITAL = 1000.0 


In [ ]:
# LOADING PRICE DATA (price per share of stock on a given day)

# Download the data for the selected ticker and parameters
price_data = yf.download(TICKER, start=START_DATE, end=END_DATE)
# Keep as multiIndex columns for now in case I want to add more tickers at same time
print(f"Columns: {price_data.columns}")
print(f"Shape: {price_data.shape}")

# Extract the Close price because I only need one price series for SMA
# Makes access easier than multi-level indexing
# Apparently using 'Close' is standard to compute SMAs
# This DataFrame contains closing price per share of stock on a given day
prices = price_data['Close'][TICKER].to_frame('price')

# Removes rows that contain missing data
prices = prices.dropna() # idk if this is really necessary to keep tho

# Print basic information about price
print(f"Loaded {len(prices)} days of data for {TICKER}")
print(f"Date range: {prices.index[0].date()} to {prices.index[-1].date()}")

# Prints first 10 rows for prices (default is 5)
prices.head(10)
#prices.tail()


In [ ]:
# SMA Crossover Rule/Strategy

# TODO: SMA Crossover(Buy when 20-day SMA > 50-day SMA, sell when 20-day SMA <= 50-day SMA)
# Crossover Strategy: Buy/Sell when 2 moving averages cross each other. 
# Crossover happens when when fast line crosses slow line.
# Entry/Buy = fast line crosses above slow line 
# Exit/Sell = fast line crosses below slow line

# Define windows for the SMAs
fast_window = 20 # 20-day SMA
slow_window = 50 # 50-day SMA

#Calculate both moving averages
data = prices.copy()
data['sma_fast'] = data['price'].rolling(window=fast_window, min_periods=fast_window).mean()
data['sma_slow'] = data['price'].rolling(window=slow_window, min_periods=slow_window).mean()

# Signals indicate when to trade: 
# 1 = buy signal (sma_fast > sma_slow), 0 = sell signal (sma_fast <= sma_slow)
data['signal'] = (data['sma_fast'] > data['sma_slow']).astype(int)
print(f"Signal Summary:")
print(f"Buy signals (1): {(data['signal'] == 1).sum()} days")
print(f"Sell signals (0): {(data['signal'] == 0).sum()} days")

# IRL, close happens at end of day, so enter tomorrow based on today's signal
# Ensures trade based on yesterday's signal, not today's
# data['signal'].shift(1) uses previous day's signal to avoid look-ahead bias
# fillna(0) fills NaN values with 0
data['position'] = data['signal'].shift(1).fillna(0).astype(int)

# Remove days where there's not enough data for SMAs
data = data.dropna()

# Print first 10 rows with signals
print(f"\nFirst 10 rows w/ signals:")
data[['price', 'sma_fast', 'sma_slow', 'signal', 'position']].head(10)


In [ ]:
# EXECUTION ENGINE

# Execution = 'buying/selling'
# Engine = 'simulator'

# Define all variables I'll need for strategy loop
funds = STARTING_CAPITAL
shares = 0
portfolio_values = []
trade_log = []

prev_position = 0

# data looks like this:
#              price  sma_fast  sma_slow  signal  position
# 2021-02-26   233.12   225.50   230.00     0        0
# 2021-02-27   235.46   226.10   229.50     1        0
for date, row in data.iterrows():
    price = row['price']
    curr_position = int(row['position'])
    
    # Check if position changed
    if curr_position != prev_position:

        # curr_position == 1 means today I am holding stock
        # prev_position == 0 means yesterday I was not holding stock
        if curr_position == 1 and prev_position == 0:
            # Buy as many shares as possible w/ available funds
            num_of_shares_to_buy = int(funds // price) # // is floor division that returns float, which rounds down to nearest integer
            if num_of_shares_to_buy > 0:
                cost = num_of_shares_to_buy * price
                funds -= cost
                shares += num_of_shares_to_buy
                trade_log.append({
                    'Date': date,
                    'Action': 'BUY',
                    'Price': price,
                    'Shares': num_of_shares_to_buy,
                    'Position': curr_position,
                    'Cash': funds,
                    # 'PortfolioValue' is total worth of all investments + funds
                    'PortfolioValue': funds + (shares * price)
                })
        

        elif curr_position == 0 and prev_position == 1:
            # Sell all shares
            if shares > 0:
                # 'proceeds' is total amount I get from selling all shares
                proceeds = shares * price
                funds += proceeds
                trade_log.append({
                    'Date': date,
                    'Action': 'SELL',
                    'Price': price,
                    'Shares': shares,
                    'Position': curr_position,
                    'Cash': funds,
                    'PortfolioValue': funds
                })
                shares = 0 # Now I have zero shares
        
        prev_position = curr_position
    
    # TODO: Track positions (1 or 0) and portfolio value over time...
    # Track portfolio value for EACH day
    portfolio_value = funds + (shares * price)
    portfolio_values.append(portfolio_value)

# Append portfolio values to data
data['portfolio_value'] = portfolio_values
# data.head()

# TODO: Output a trade log (Date, Action, Price, Position, PortfolioValue)...
# Create trade log DataFrame with numbered rows
trade_log_data_frame = pd.DataFrame(trade_log)
if not trade_log_data_frame.empty:
    trade_log_data_frame.index = range(1, len(trade_log_data_frame) + 1)
    trade_log_data_frame.index.name = 'Trade #'

print(f"\nTrade Log:")
trade_log_data_frame
